# ツイートを学習して文章を生成
参考：https://klab.hateblo.jp/entry/2021/11/02/173559

## 準備
tweetフォルダを作成し、その中にテキストファイル（学習データ）を入れておく。
### 例）tweet/tweets.txtの上三行
```
ミックスダウンやり直しにつき<n>本日も無し。<n><n>徹頭徹尾無し！<|endoftext|>
ミックスダウン闌につき<n>本日も無し。<n><n>全身全霊で無し。<|endoftext|>
レコーディング闌につき、本日のTWは無し。<|endoftext|>
```

## GPT-2をインストール

In [1]:
# gpt2-japaneseのインストール

!git clone https://github.com/tanreinama/gpt2-japanese

Cloning into 'gpt2-japanese'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 269 (delta 125), reused 157 (delta 66), pack-reused 41
Receiving objects: 100% (269/269), 1.22 MiB | 3.10 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [2]:
%cd gpt2-japanese

/notebooks/gpt2-japanese


In [3]:
!pip uninstall tensorflow -y

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 497.6 MB 19.4 MB/s eta 0:00:01    |█▋                              | 25.5 MB 20.7 MB/s eta 0:00:23     |██                              | 30.4 MB 20.7 MB/s eta 0:00:23     |██████████████████████▊         | 352.5 MB 5.0 MB/s eta 0:00:30
     |████████████████████████████████| 2.1 MB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 46.0 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 33.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 46.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 50.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 29.8 MB/s eta 0:00:01
  Created wheel for jaconv: file

## モデルをダウンロード

In [5]:
!wget https://www.nama.ne.jp/models/gpt2ja-small.tar.bz2

--2022-04-22 07:11:24--  https://www.nama.ne.jp/models/gpt2ja-small.tar.bz2
Resolving www.nama.ne.jp (www.nama.ne.jp)... 112.78.112.176
Connecting to www.nama.ne.jp (www.nama.ne.jp)|112.78.112.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 386498483 (369M) [application/x-bzip2]
Saving to: ‘gpt2ja-small.tar.bz2’

gpt2ja-small.tar.bz 100%[===================>] 368.59M  2.39MB/s    in 4m 45s  

2022-04-22 07:16:11 (1.29 MB/s) - ‘gpt2ja-small.tar.bz2’ saved [386498483/386498483]



In [6]:
!tar xvfj gpt2ja-small.tar.bz2

gpt2ja-small/
gpt2ja-small/checkpoint
gpt2ja-small/model-10412700.data-00000-of-00001
gpt2ja-small/model-10412700.index
gpt2ja-small/model-10412700.meta
special_thanks.txt


## データセットを作成

### gpt2-japanese直下にtweetフォルダを移動

In [7]:
!mv ../tweet ./

In [8]:
# finetune.npzに変換
!git clone https://github.com/tanreinama/Japanese-BPEEncoder.git

Cloning into 'Japanese-BPEEncoder'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 37 (delta 16), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (37/37), 283.21 KiB | 297.00 KiB/s, done.


In [9]:
!python ./Japanese-BPEEncoder/encode_bpe.py --src_dir tweet --dst_file finetune

append # tweet
append # tweet/.ipynb_checkpoints
0it [00:00, ?it/s]                                        | 0/1 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  4.44it/s]


## ファインチューニング
run_finetune.pyを以下のように書き換える。

```py:run_finetune.py
        try:
           　for i in range(300): # for文に変更
           　# while True: # 無限ループをコメントアウト
           
                〜省略〜
               
            save() # 保存を追加
        except KeyboardInterrupt:
            print('interrupted')
            save()
```

In [12]:
!python run_finetune.py --base_model gpt2ja-small --dataset finetune.npz --run_name gpr2ja-finetune_run1

2022-04-22 07:31:14.962102: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-22 07:31:14.996927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 07:31:15.076183: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 07:31:15.076793: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

## モデルで文章を生成
gpt2-generate.pyをダウンロードして以下のように書き換える。

```py:gpt2-generate.py
ckpt = tf.train.latest_checkpoint("checkpoint/gpr2ja-finetune_run1")
```

In [15]:
!python gpt2-generate.py --model checkpoint/gpr2ja-finetune_run1-small --num_generate 40

2022-04-22 07:46:02.914911: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-22 07:46:02.943533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 07:46:03.000569: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 07:46:03.001090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer